# Tara Sequencer

In [1]:
# imports
from importlib import reload
import numpy as np
import pandas
import datetime

from matplotlib import pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy

import seaborn as sns

from oceancolor.tara import io
from oceancolor.tara import spectra

import sequencer
from sequencer import sequencer_

# Load the Table

In [2]:
tara_db = io.load_tara_db()

# Process the spectra

In [3]:
reload(spectra)
wv_nm, all_a_ph, all_a_ph_sig = spectra.spectra_from_table(tara_db)
wv_grid = np.arange(402.5, 707.5, 5.)
rwv_nm, r_aph, r_sig = spectra.rebin_to_grid(wv_nm, all_a_ph, all_a_ph_sig, wv_grid)

/home/xavier/Projects/Oceanography/python/ocean-color/oceancolor/tara/spectra.py:248: RuntimeWarning: invalid value encountered in divide
  isum = np.nansum(values[gd]*mask[gd], axis=0) / np.sum(mask[gd],axis=0)
/home/xavier/Projects/Oceanography/python/ocean-color/oceancolor/tara/spectra.py:249: RuntimeWarning: invalid value encountered in divide
  esum = np.nansum(err_vals[gd]*mask[gd], axis=0) / np.sum(mask[gd],axis=0)


In [4]:
r_aph.shape

(302779, 60)

# Remove bad spectra

In [5]:
tot_spec = np.nansum(r_aph, axis=-1)
gd_tot = tot_spec > 0.

## cut

In [6]:
cull_raph = r_aph[gd_tot, :]
cull_rsig = r_sig[gd_tot, :]

# Deal with NaN

In [7]:
bad = np.isnan(cull_raph) | (cull_rsig <= 0.) | (cull_raph <0.)
print(f'Nbad = {np.sum(bad)}')
# Replace
cull_raph[bad] = 0.
cull_rsig[bad] = 1e5

Nbad = 81851


# Start with a random set of spectra

In [8]:
rand = np.random.choice(np.arange(cull_raph.shape[0]), size=100, replace=False)
rand_raph = cull_raph[rand,:]

# Init the Sequencer

In [9]:
reload(sequencer_)
estimator_list = ['EMD', 'energy', 'L2'] # Removed KL because of the 0 values
seq = sequencer.Sequencer(rwv_nm, rand_raph, estimator_list)

# Run it

## Output directory

In [10]:
# to execute the sequencer, we first need to define the output directory to which the different 
# outputs will be saved
output_path = "/home/xavier/Projects/Oceanography/Color/Sequencer/Tara"
final_elongation, final_sequence = seq.execute(output_path)

calculating the distance matrices for estimator: EMD, scale: 1
finished calculating this distance matrix list, it took: 0.309450626373291 seconds
calculating the distance matrices for estimator: EMD, scale: 2
finished calculating this distance matrix list, it took: 0.4994215965270996 seconds
calculating the distance matrices for estimator: energy, scale: 1
finished calculating this distance matrix list, it took: 0.269620418548584 seconds
calculating the distance matrices for estimator: energy, scale: 2
finished calculating this distance matrix list, it took: 0.5101320743560791 seconds
calculating the distance matrices for estimator: L2, scale: 1
finished calculating this distance matrix list, it took: 0.023654460906982422 seconds
calculating the distance matrices for estimator: L2, scale: 2
finished calculating this distance matrix list, it took: 0.04581856727600098 seconds
dumped the distance matrix dictionaries to the file: /home/xavier/Projects/Oceanography/Color/Sequencer/Tara/dist

/home/xavier/Python/Sequencer/sequencer/sequencer_.py:1032: RuntimeWarning: divide by zero encountered in divide
  distance_matrix = 1.0 / proximity_matrix_copy


----

# Debuggin

In [23]:
norm = np.sum(r_aph, axis=-1)
norm.size

302779

In [25]:
bad = norm <= 0.
np.sum(bad)

15